In [ ]:
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Polygon

# Input parameters
Name = 'Yvetot'
xMin = 0.6
xMax = 0.86
yMin = 49.67
yMax = 49.57
projected_crs = 'EPSG:2154'

# Create bounding box polygon
lon_point_list = [xMin, xMax, xMax, xMin]
lat_point_list = [yMax, yMax, yMin, yMin]
polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon_geom])

# Output Geopackage name
gpkg = f'SDM24_{Name}_raw.gpkg'

# Function to clean data
def clean_geodataframe(gdf):
    # Keep only polygon geometries
    gdf = gdf[gdf.geometry.type == 'Polygon']
    # Drop columns with list values
    list_columns = [col for col in gdf.columns if gdf[col].apply(lambda x: isinstance(x, list)).any()]
    gdf = gdf.drop(columns=list_columns, errors='ignore')
    return gdf

# 1. Download OSM buildings
print("Step 1: Downloading OSM buildings.")
buildings = ox.features.features_from_polygon(polygon['geometry'][0], tags={'building': True})

# Clean buildings data
print("Cleaning buildings data.")
buildings = clean_geodataframe(buildings)
buildings = buildings.to_crs(projected_crs)

# Save buildings to Geopackage
buildings.to_file(gpkg, layer='osm_all_buildings3', driver="GPKG")

# 2. Download OSM streets
print("Step 2: Downloading OSM streets.")
# Extract the streets based on the bounding box
G = ox.graph_from_polygon(polygon['geometry'][0], network_type='all')

# Convert graph to GeoDataFrame
print("Step 2.1: Converting streets graph to GeoDataFrame.")
linestrings_df = ox.convert.graph_to_gdfs(G)
linestrings_df = linestrings_df[1]
linestrings_gdf = gpd.GeoDataFrame(linestrings_df, geometry='geometry')

# Function to convert lists to strings
def convert_list_to_string(val):
    if isinstance(val, list):
        return ', '.join(map(str, val))
    return val

# Call the above function
print("Step 2.2: Converting lists to strings.")
for column in linestrings_gdf.columns:
    if any(isinstance(val, list) for val in linestrings_gdf[column]):
        linestrings_gdf[column] = linestrings_gdf[column].apply(convert_list_to_string)

# Keep essential columns only if they exist
essential_columns = ['geometry', 'osmid', 'highway','oneway','maxspeed','reversed','lanes','width','access','service','tunnel','bridge','junction']
existing_columns = linestrings_gdf.columns.intersection(essential_columns)
linestrings_gdf = linestrings_gdf.loc[:, existing_columns]

linestrings_gdf = linestrings_gdf.to_crs(projected_crs)
linestrings_gdf.to_file(gpkg, layer='osm_all_streets5', driver="GPKG")

print("Process complete. Raw OSM data saved.")